In [6]:
import math
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter
import numpy as np
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict
from sklearn.base import TransformerMixin


## Data  Cleaning

In [2]:
data = pd.read_excel("Data SA.xlsx")
data.head()

,created_at,full_text,tweet_url,username,sentimen
0,Sun Aug 04 00:51:01 +0000 2024,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,https://x.com/qwenvy/status/1819898803652366643,qwenvy,negatif
1,Mon Aug 05 06:10:13 +0000 2024,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,https://x.com/turfaul/status/1820341521125875955,turfaul,netral
2,Fri Aug 02 13:23:29 +0000 2024,Terimakasih @GrabID https://t.co/t3xsXonkg4,https://x.com/Zanboy04/status/1819363393100566662,Zanboy04,positif
3,Wed Aug 07 01:29:41 +0000 2024,Pagi ini @GrabID kok gabisa klik motor atau mo...,https://x.com/sipietz/status/1820995699078783021,sipietz,netral
4,Wed Jul 31 15:20:36 +0000 2024,makasih banyak fiturnya by... @GrabID https://...,https://x.com/moment4lifers/status/18186680915...,moment4lifers,positif


In [3]:
import re
import string
import nltk
import emoji

# Fungsi untuk menghapus URL
def remove_URL(tweet):
    if tweet is not None and isinstance(tweet, str):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'', tweet)
    else:
        return tweet

# Fungsi untuk menghapus HTML
def remove_html(tweet):
    if tweet is not None and isinstance(tweet, str):
        html = re.compile(r'<.*?>')
        return html.sub(r'', tweet)
    else:
        return tweet

# Fungsi untuk menghapus angka
def remove_numbers(tweet):
    if tweet is not None and isinstance(tweet, str):
        tweet = re.sub(r'\d', '', tweet)  # Menghapus semua angka
    return tweet

def remove_username(text):
    import re
    return re.sub(r'@[^\s]+', '', text)

# Mapping emotikon ke teks dalam bentuk string literal (bukan pattern regex)
emot_map = {
    ":-)": "senyum",
    ":)": "senyum",
    ":-D": "tertawa",
    ":D": "tertawa",
    ":'\)": "menangis bahagia",
    ":-(": "sedih",
    ":(": "sedih",
    ":'‑(": "menangis",
    ":'(": "menangis",
    ":‑O": "terkejut",
    ":O": "terkejut",
    ":|": "datar",
    ":-/": "bingung",
    ":/": "bingung",
    ":-*": "cium",
    ":*": "cium",
    ";-)": "kedipan mata",
    ";)": "kedipan mata",
    ">:(": "marah",
    ":P": "menjulurkan lidah",
    ":-P": "menjulurkan lidah",
    ":3": "imut",
    "-_-": "kesal",
    "o_O": "bingung",
    "O_o": "bingung",
    ">.<": "frustrasi",
    "<3": "cinta",
    "</3": "patah hati",
}

# Escape semua key agar bisa digunakan dalam regex dengan aman
escaped_keys = list(map(re.escape, emot_map.keys()))
emot_regex = re.compile("|".join(escaped_keys))

# Fungsi untuk konversi
def convert_emoticon(text):
    text = emot_regex.sub(lambda m: emot_map[m.group()], str(text))  # gunakan str() untuk menghindari error NoneType
    text = emoji.demojize(text)
    return text.replace(":", "")

data['cleaning'] = data['full_text'].apply(lambda x: remove_URL(x))
data['cleaning'] = data['cleaning'].apply(lambda x: remove_username(x))
data['cleaning'] = data['cleaning'].apply(lambda x: remove_html(x))
data['cleaning'] = data['cleaning'].apply(lambda x: remove_numbers(x))
data['cleaning'] = data['cleaning'].apply(lambda x: convert_emoticon(x))

data.head(5)

,created_at,full_text,tweet_url,username,sentimen,cleaning
0,Sun Aug 04 00:51:01 +0000 2024,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,https://x.com/qwenvy/status/1819898803652366643,qwenvy,negatif,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...
1,Mon Aug 05 06:10:13 +0000 2024,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,https://x.com/turfaul/status/1820341521125875955,turfaul,netral,Diskon
2,Fri Aug 02 13:23:29 +0000 2024,Terimakasih @GrabID https://t.co/t3xsXonkg4,https://x.com/Zanboy04/status/1819363393100566662,Zanboy04,positif,Terimakasih
3,Wed Aug 07 01:29:41 +0000 2024,Pagi ini @GrabID kok gabisa klik motor atau mo...,https://x.com/sipietz/status/1820995699078783021,sipietz,netral,Pagi ini kok gabisa klik motor atau mobil ya?
4,Wed Jul 31 15:20:36 +0000 2024,makasih banyak fiturnya by... @GrabID https://...,https://x.com/moment4lifers/status/18186680915...,moment4lifers,positif,makasih banyak fiturnya by...


In [4]:
!pip install emoji
!pip install translate

import emoji
from translate import Translator
import pandas as pd
import re

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
def emoji_to_alias(text):
    return emoji.demojize(text)

def translate_alias_to_indonesian(text):
    translator = Translator(to_lang="id")
    words = re.split(r'(:[^:]+:)', text)
    translated_words = []
    for word in words:
        if word.startswith(':') and word.endswith(':'):
            try:
                word_clean = word.strip(':').replace('_', ' ')
                translated = translator.translate(word_clean)
                translated_words.append(translated)
            except:
                translated_words.append(word_clean)
        else:
            translated_words.append(word)
    return ' '.join(translated_words)


def clean_and_translate(text):
    alias_text = emoji_to_alias(text)
    return translate_alias_to_indonesian(alias_text)

data['cleaning'] = data['cleaning'].apply(lambda x:clean_and_translate(x))
data.head()

,created_at,full_text,tweet_url,username,sentimen,cleaning
0,Sun Aug 04 00:51:01 +0000 2024,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,https://x.com/qwenvy/status/1819898803652366643,qwenvy,negatif,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...
1,Mon Aug 05 06:10:13 +0000 2024,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,https://x.com/turfaul/status/1820341521125875955,turfaul,netral,Diskon
2,Fri Aug 02 13:23:29 +0000 2024,Terimakasih @GrabID https://t.co/t3xsXonkg4,https://x.com/Zanboy04/status/1819363393100566662,Zanboy04,positif,Terimakasih
3,Wed Aug 07 01:29:41 +0000 2024,Pagi ini @GrabID kok gabisa klik motor atau mo...,https://x.com/sipietz/status/1820995699078783021,sipietz,netral,Pagi ini kok gabisa klik motor atau mobil ya?
4,Wed Jul 31 15:20:36 +0000 2024,makasih banyak fiturnya by... @GrabID https://...,https://x.com/moment4lifers/status/18186680915...,moment4lifers,positif,makasih banyak fiturnya by...


In [6]:
def case_folding(text):
    if isinstance(text, str):
        lowercase_text = text.lower()
        return lowercase_text
    else:
        return text

data['case_folding'] = data['cleaning'].apply(case_folding)
data.head(5)

,created_at,full_text,tweet_url,username,sentimen,cleaning,case_folding
0,Sun Aug 04 00:51:01 +0000 2024,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,https://x.com/qwenvy/status/1819898803652366643,qwenvy,negatif,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,shock bgt pagi²...woy ini rumah gw di pantau m...
1,Mon Aug 05 06:10:13 +0000 2024,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,https://x.com/turfaul/status/1820341521125875955,turfaul,netral,Diskon,diskon
2,Fri Aug 02 13:23:29 +0000 2024,Terimakasih @GrabID https://t.co/t3xsXonkg4,https://x.com/Zanboy04/status/1819363393100566662,Zanboy04,positif,Terimakasih,terimakasih
3,Wed Aug 07 01:29:41 +0000 2024,Pagi ini @GrabID kok gabisa klik motor atau mo...,https://x.com/sipietz/status/1820995699078783021,sipietz,netral,Pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok gabisa klik motor atau mobil ya?
4,Wed Jul 31 15:20:36 +0000 2024,makasih banyak fiturnya by... @GrabID https://...,https://x.com/moment4lifers/status/18186680915...,moment4lifers,positif,makasih banyak fiturnya by...,makasih banyak fiturnya by...


In [7]:
import pandas as pd

# Fungsi penggantian kata tidak baku
def replace_taboo_words(text, kamus_tidak_baku):
    if isinstance(text, str):
        words = text.split()
        replaced_words = []
        kalimat_baku = []
        kata_diganti = []
        kata_tidak_baku_hash = []

        for word in words:
            if word in kamus_tidak_baku:
                baku_word = kamus_tidak_baku[word]
                if isinstance(baku_word, str) and all(char.isalpha() for char in baku_word):
                    replaced_words.append(baku_word)
                    kalimat_baku.append(baku_word)
                    kata_diganti.append(word)
                    kata_tidak_baku_hash.append(hash(word))
                # else:
                #     replaced_words.append('<karakter eksplisit>')
                #     kalimat_baku.append(word)
            else:
                replaced_words.append(word)
        replaced_text = ' '.join(replaced_words)
    else:
        replaced_text = ''
        kalimat_baku = []
        kata_diganti = []
        kata_tidak_baku_hash = []

    return replaced_text, kalimat_baku, kata_diganti, kata_tidak_baku_hash

In [10]:
# Baca dataset
data_nor =  pd.DataFrame(data[['created_at','username','sentimen','full_text','cleaning','case_folding']])
# data.head(5)

# Baca kamus kata tidak baku
kamus_data = pd.read_excel('kamuskatabaku.xlsx')
kamus_tidak_baku = dict(zip(kamus_data['tidak_baku'], kamus_data['kata_baku']))

In [11]:
# Terapkan fungsi penggantian kata tidak baku
data_nor['normalisasi'], data_nor['Kata_Baku'], data_nor['Kata_Tidak_Baku'], data_nor['Kata_Tidak_Baku_Hash'] = zip(*data_nor['case_folding'].apply(lambda x: replace_taboo_words(x, kamus_tidak_baku)))
# data_nor.head(100)

data =  pd.DataFrame(data_nor[['created_at','username','sentimen','full_text','cleaning','case_folding','normalisasi']])

data.head(5)

,created_at,username,sentimen,full_text,cleaning,case_folding,normalisasi
0,Sun Aug 04 00:51:01 +0000 2024,qwenvy,negatif,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,shock bgt pagi²...woy ini rumah gw di pantau m...,shock banget pagi²...woy ini rumah gue di pant...
1,Mon Aug 05 06:10:13 +0000 2024,turfaul,netral,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,Diskon,diskon,diskon
2,Fri Aug 02 13:23:29 +0000 2024,Zanboy04,positif,Terimakasih @GrabID https://t.co/t3xsXonkg4,Terimakasih,terimakasih,terimakasih
3,Wed Aug 07 01:29:41 +0000 2024,sipietz,netral,Pagi ini @GrabID kok gabisa klik motor atau mo...,Pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok klik motor atau mobil ya?
4,Wed Jul 31 15:20:36 +0000 2024,moment4lifers,positif,makasih banyak fiturnya by... @GrabID https://...,makasih banyak fiturnya by...,makasih banyak fiturnya by...,banyak fiturnya by...


In [12]:
def tokenize(text):
    tokens = text.split()
    return tokens

data['tokenize'] = data['normalisasi'].apply(tokenize)

data.head(5)

,created_at,username,sentimen,full_text,cleaning,case_folding,normalisasi,tokenize
0,Sun Aug 04 00:51:01 +0000 2024,qwenvy,negatif,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,shock bgt pagi²...woy ini rumah gw di pantau m...,shock banget pagi²...woy ini rumah gue di pant...,"[shock, banget, pagi²...woy, ini, rumah, gue, ..."
1,Mon Aug 05 06:10:13 +0000 2024,turfaul,netral,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,Diskon,diskon,diskon,[diskon]
2,Fri Aug 02 13:23:29 +0000 2024,Zanboy04,positif,Terimakasih @GrabID https://t.co/t3xsXonkg4,Terimakasih,terimakasih,terimakasih,[terimakasih]
3,Wed Aug 07 01:29:41 +0000 2024,sipietz,netral,Pagi ini @GrabID kok gabisa klik motor atau mo...,Pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok klik motor atau mobil ya?,"[pagi, ini, kok, klik, motor, atau, mobil, ya?]"
4,Wed Jul 31 15:20:36 +0000 2024,moment4lifers,positif,makasih banyak fiturnya by... @GrabID https://...,makasih banyak fiturnya by...,makasih banyak fiturnya by...,banyak fiturnya by...,"[banyak, fiturnya, by...]"


In [13]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to C:\Users\Cut Azimah -
[nltk_data]     Iffah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def remove_stopwords(tokens):
    return [t for t in tokens if t not in stop_words]
data['stopword removal'] = data['tokenize'].apply(remove_stopwords)

data.head(5)

,created_at,username,sentimen,full_text,cleaning,case_folding,normalisasi,tokenize,stopword removal
0,Sun Aug 04 00:51:01 +0000 2024,qwenvy,negatif,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,shock bgt pagi²...woy ini rumah gw di pantau m...,shock banget pagi²...woy ini rumah gue di pant...,"[shock, banget, pagi²...woy, ini, rumah, gue, ...","[shock, banget, pagi²...woy, rumah, gue, panta..."
1,Mon Aug 05 06:10:13 +0000 2024,turfaul,netral,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,Diskon,diskon,diskon,[diskon],[diskon]
2,Fri Aug 02 13:23:29 +0000 2024,Zanboy04,positif,Terimakasih @GrabID https://t.co/t3xsXonkg4,Terimakasih,terimakasih,terimakasih,[terimakasih],[terimakasih]
3,Wed Aug 07 01:29:41 +0000 2024,sipietz,netral,Pagi ini @GrabID kok gabisa klik motor atau mo...,Pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok klik motor atau mobil ya?,"[pagi, ini, kok, klik, motor, atau, mobil, ya?]","[pagi, klik, motor, mobil, ya?]"
4,Wed Jul 31 15:20:36 +0000 2024,moment4lifers,positif,makasih banyak fiturnya by... @GrabID https://...,makasih banyak fiturnya by...,makasih banyak fiturnya by...,banyak fiturnya by...,"[banyak, fiturnya, by...]","[fiturnya, by...]"


In [15]:
!pip install Sastrawi

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

Defaulting to user installation because normal site-packages is not writeable


In [16]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(tokens):
    return [stemmer.stem(word) for word in tokens]

data['steming_data'] = data['stopword removal'].apply(lambda x: ' '.join(stem_text(x)))
data.head(10)

,created_at,username,sentimen,full_text,cleaning,case_folding,normalisasi,tokenize,stopword removal,steming_data
0,Sun Aug 04 00:51:01 +0000 2024,qwenvy,negatif,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,SHOCK BGT PAGI²...WOY INI RUMAH GW DI PANTAU M...,shock bgt pagi²...woy ini rumah gw di pantau m...,shock banget pagi²...woy ini rumah gue di pant...,"[shock, banget, pagi²...woy, ini, rumah, gue, ...","[shock, banget, pagi²...woy, rumah, gue, panta...",shock banget pagi woy rumah gue pantau maling ...
1,Mon Aug 05 06:10:13 +0000 2024,turfaul,netral,Diskon 6000 @GrabID? https://t.co/3aoxrdWnK4,Diskon,diskon,diskon,[diskon],[diskon],diskon
2,Fri Aug 02 13:23:29 +0000 2024,Zanboy04,positif,Terimakasih @GrabID https://t.co/t3xsXonkg4,Terimakasih,terimakasih,terimakasih,[terimakasih],[terimakasih],terimakasih
3,Wed Aug 07 01:29:41 +0000 2024,sipietz,netral,Pagi ini @GrabID kok gabisa klik motor atau mo...,Pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok gabisa klik motor atau mobil ya?,pagi ini kok klik motor atau mobil ya?,"[pagi, ini, kok, klik, motor, atau, mobil, ya?]","[pagi, klik, motor, mobil, ya?]",pagi klik motor mobil ya
4,Wed Jul 31 15:20:36 +0000 2024,moment4lifers,positif,makasih banyak fiturnya by... @GrabID https://...,makasih banyak fiturnya by...,makasih banyak fiturnya by...,banyak fiturnya by...,"[banyak, fiturnya, by...]","[fiturnya, by...]",fiturnya by
5,Sat Aug 03 06:38:11 +0000 2024,gabrieldyas39,positif,Promo @GrabID nggk kaleng2 diskon 55%. 50k cum...,Promo nggk kaleng diskon %. k cuman byr k.,promo nggk kaleng diskon %. k cuman byr k.,promo tidak kaleng diskon %. ke cuman byr k.,"[promo, tidak, kaleng, diskon, %., ke, cuman, ...","[promo, kaleng, diskon, %., cuman, byr, k.]",promo kaleng diskon cuman byr k
6,Wed Aug 07 11:42:08 +0000 2024,myvalentine61,netral,Yg dijemput cewe only kah? @GrabID https://t.c...,Yg dijemput cewe only kah?,yg dijemput cewe only kah?,yang dijemput cewek only kah?,"[yang, dijemput, cewek, only, kah?]","[dijemput, cewek, only, kah?]",jemput cewek only kah
7,Fri Aug 09 23:31:47 +0000 2024,MrBeemss,negatif,Emang boleh ya ngakses card tanpa pin gini? Ya...,Emang boleh ya ngakses card tanpa pin gini? Ya...,emang boleh ya ngakses card tanpa pin gini? ya...,memang boleh ya ngakses card tanpa pin gini? y...,"[memang, boleh, ya, ngakses, card, tanpa, pin,...","[ya, ngakses, card, pin, gini?, ya, maksudnya,...",ya ngakses card pin gin ya maksud pakai metode...
8,Fri Aug 09 04:00:01 +0000 2024,Beemoa2,negatif,Grab km petanya aneh bgt sumpahhhh driver gw s...,Grab km petanya aneh bgt sumpahhhh driver gw s...,grab km petanya aneh bgt sumpahhhh driver gw s...,grab kamu petanya aneh banget sumpahhhh driver...,"[grab, kamu, petanya, aneh, banget, sumpahhhh,...","[grab, petanya, aneh, banget, sumpahhhh, drive...",grab peta aneh banget sumpahhhh driver gue nya...
9,Thu Aug 08 03:22:06 +0000 2024,Ropian_Taloko,netral,dah 3 jam ini blom bunyi bunyi min gimana ini ...,dah jam ini blom bunyi bunyi min gimana ini ...,dah jam ini blom bunyi bunyi min gimana ini ...,sudah jam ini belum bunyi bunyi min bagaimana ...,"[sudah, jam, ini, belum, bunyi, bunyi, min, ba...","[jam, bunyi, bunyi, min, ordernya]",jam bunyi bunyi min order


In [17]:
 data.to_excel('dataset_15_06.xlsx',index=False)

## Dropna data

In [18]:
data = pd.read_excel('dataset_15_06.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   created_at        1140 non-null   object
 1   username          1140 non-null   object
 2   sentimen          1140 non-null   object
 3   full_text         1140 non-null   object
 4   cleaning          1140 non-null   object
 5   case_folding      1140 non-null   object
 6   normalisasi       1138 non-null   object
 7   tokenize          1140 non-null   object
 8   stopword removal  1140 non-null   object
 9   steming_data      1134 non-null   object
dtypes: object(10)
memory usage: 89.2+ KB


In [19]:
df = data.dropna

In [20]:
data = df()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1134 entries, 0 to 1139
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   created_at        1134 non-null   object
 1   username          1134 non-null   object
 2   sentimen          1134 non-null   object
 3   full_text         1134 non-null   object
 4   cleaning          1134 non-null   object
 5   case_folding      1134 non-null   object
 6   normalisasi       1134 non-null   object
 7   tokenize          1134 non-null   object
 8   stopword removal  1134 non-null   object
 9   steming_data      1134 non-null   object
dtypes: object(10)
memory usage: 97.5+ KB


In [21]:
 data.to_excel('dataset_15_06_bersih.xlsx',index=False)

## label encoder

In [22]:
from sklearn.preprocessing import LabelEncoder

data = pd.read_excel('dataset_15_06_bersih.xlsx')
# Inisialisasi LabelEncoder
encoder = LabelEncoder()
data['label'] = encoder.fit_transform(data['sentimen'])

#Melihat mapping
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("\nLabel mapping:", label_mapping)
print(data['label'].value_counts())



Label mapping: {'negatif': 0, 'netral': 1, 'positif': 2}
label
0    541
1    460
2    133
Name: count, dtype: int64


In [23]:
 data.to_excel('dataset_15_06_label_2.xlsx',index=False)

## UJICOBA  2 - FINAL

🔧 1. Hitung QER untuk 3 kelas (rata-rata antar kelas)

In [7]:
data = pd.read_excel('dataset_15_06_label_2.xlsx')

In [8]:
def compute_qer_scores_multi_class_avg(data, label_col='label', token_col='steming_data'):
    from collections import defaultdict
    import math

    doc_count = defaultdict(lambda: defaultdict(int))
    class_counts = defaultdict(int)

    for tokens, label in zip(data[token_col], data[label_col]):
        class_counts[label] += 1
        for token in set(tokens):  # LANGSUNG gunakan list token
            doc_count[label][token] += 1

    all_tokens = set()
    for c in doc_count:
        all_tokens.update(doc_count[c].keys())

    classes = list(class_counts.keys())
    qer_scores = {}

    for token in all_tokens:
        p = {}
        for c in classes:
            p[c] = (doc_count[c].get(token, 0) + 1) / (class_counts[c] + 1)

        avg_scores = []
        for i, c in enumerate(classes):
            others = [oc for oc in classes if oc != c]
            other_probs = sum([p[oc] for oc in others]) / len(others)
            if other_probs == 0:
                other_probs = 1e-6
            score = abs(math.log(p[c] / other_probs))
            avg_scores.append(score)

        qer_scores[token] = sum(avg_scores) / len(avg_scores)

    return qer_scores


🧪 3. Fungsi evaluasi semua eksperimen:

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from scipy.sparse import diags
import numpy as np
from collections import Counter
import ast

# # Ubah hanya jika masih string
data['steming_data'] = data['steming_data'].apply(
    lambda x: x.split() if isinstance(x, str) else x
)

# # Pastikan 'steming_data' adalah list of token
# data['filtered_text'] = data['steming_data'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

data['filtered_text'] = data['steming_data'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
def evaluate_experiment_3_safe_pipeline(data, top_k=150, alpha=0.5):
    X_raw_all = data['filtered_text']         # hasil join dari list token
    tokens_data_all = data['steming_data']    # list of tokens
    y_all = data['label']

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_y_true, all_y_pred = [], []
    top_tokens_per_fold = []
    qer_scores_per_fold = []
    fold_accuracies = []

    best_model = None
    best_acc = 0
    best_vectorizer = None
    best_qer_scores = None


    for fold, (train_idx, test_idx) in enumerate(skf.split(X_raw_all, y_all), 1):
        print(f"Fold {fold} dimulai...")

        print(f"  Train size: {len(train_idx)}, Test size: {len(test_idx)}")

        # Split sesuai indeks
        X_train_raw = X_raw_all.iloc[train_idx]
        X_test_raw = X_raw_all.iloc[test_idx]
        tokens_train = tokens_data_all.iloc[train_idx].tolist()
        y_train = y_all.iloc[train_idx]
        y_test = y_all.iloc[test_idx]

        # Siapkan DataFrame untuk training subset
        train_data = data.iloc[train_idx].copy()

        # Hitung QER dari data training SAJA
        qer_scores = compute_qer_scores_multi_class_avg(
            train_data, label_col='label', token_col='steming_data'
        )
        top_tokens = sorted(qer_scores, key=qer_scores.get, reverse=True)[:top_k]

        # Vectorizer dari data training SAJA
        vectorizer = TfidfVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train_raw)
        X_test_vec = vectorizer.transform(X_test_raw)

        # --- Tampilkan Top 10 IDF Terms ---
        feature_names = vectorizer.get_feature_names_out()
        idf_values = vectorizer.idf_
        idf_df = pd.DataFrame(list(zip(feature_names, idf_values)), columns=['Word', 'IDF'])
        top_10_idf = idf_df.sort_values(by='IDF', ascending=False).head(10)

        # Hitung bobot QER berdasarkan vocab vectorizer
        feature_names = vectorizer.get_feature_names_out()
        qer_weights = np.array([qer_scores.get(word, 1.0) for word in feature_names])
        qer_weights = qer_weights ** alpha  # <<< Tuning di sini
        W = diags(qer_weights)
        X_train_vec = X_train_vec @ W
        X_test_vec = X_test_vec @ W

        # Oversampling dengan SMOTE
        smote = SMOTE(random_state=42, sampling_strategy='not majority')
        X_train_res, y_train_res = smote.fit_resample(X_train_vec, y_train)

        # Latih model
        model = SVC(C=1, kernel='linear', probability=True, random_state=42,  class_weight='balanced')
        model.fit(X_train_res, y_train_res)

        y_pred = model.predict(X_test_vec)
        acc = accuracy_score(y_test, y_pred)
        print(f"Akurasi Fold {fold}: {acc:.4f}")
        fold_accuracies.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_model = model
            best_vectorizer = vectorizer
            best_qer_scores = qer_scores

        # Prediksi & simpan hasil
        all_y_true.extend(y_test)
        all_y_pred.extend(y_pred)

        top_tokens_per_fold.append(top_tokens)
        qer_scores_per_fold.append(qer_scores)

        print(f"Fold {fold} selesai.")
    
     # === Simpan Model Terbaik ===
    # print("\nFold terbaik disimpan dengan akurasi: {:.4f}".format(best_acc))
    # joblib.dump(best_model, "model_svm_qer_best.pkl")
    # joblib.dump(best_vectorizer, "vectorizer_tfidf_best.pkl")
    # joblib.dump(best_qer_scores, "qer_scores_best.pkl")

    # Evaluasi akhir
    print("\n=== Classification Report ===")
    print(classification_report(all_y_true, all_y_pred, target_names=['Kelas 0', 'Kelas 1', 'Kelas 2']))
    print("=== Confusion Matrix ===")
    print(confusion_matrix(all_y_true, all_y_pred))

     # Hitung Cohen's Kappa
    kappa = cohen_kappa_score(all_y_true, all_y_pred)
    print(f"Cohen's Kappa Score: {kappa:.4f}")
    print(f"Akurasi Rata-rata: {np.mean(np.array(all_y_true) == np.array(all_y_pred)):.4f}")

    #   # Hitung akurasi per fold
    # acc = accuracy_score(y_test, y_pred)
    # fold_accuracies.append(acc)
    # print(f"Akurasi Fold {fold}: {acc:.4f}")

        # Rata-rata akurasi semua fold
    mean_acc = np.mean(fold_accuracies)
    print(f"\nAkurasi Rata-rata dari Semua Fold: {mean_acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_model = model
        best_vectorizer = vectorizer
        best_qer_scores = qer_scores


     # Tambahkan return agar bisa diakses dari luar fungsi
    # return all_y_true, all_y_pred, X_raw_all, top_tokens, qer_scores, top_tokens_per_fold, qer_scores_per_fold,  kappa
    return all_y_true, all_y_pred, X_raw_all, top_tokens, qer_scores, top_tokens_per_fold, qer_scores_per_fold, kappa, best_model, best_vectorizer, mean_acc


In [25]:
# all_y_true, all_y_pred, top_tokens, qer_scores = evaluate_experiment_3_safe_pipeline(data)
# all_y_true, all_y_pred, texts, top_tokens, qer_scores, top_tokens_per_fold, qer_scores_per_fold,  kappa = evaluate_experiment_3_safe_pipeline(data, alpha=0.5)
(
    y_true,
    y_pred,
    X_all,
    last_top_tokens,
    last_qer_scores,
    top_tokens_all_folds,
    qer_scores_all_folds,
    kappa,
    best_model,
    best_vectorizer,
    mean_accuracy
) = evaluate_experiment_3_safe_pipeline(data)


Fold 1 dimulai...
  Train size: 907, Test size: 227
Akurasi Fold 1: 0.7489
Fold 1 selesai.
Fold 2 dimulai...
  Train size: 907, Test size: 227
Akurasi Fold 2: 0.7181
Fold 2 selesai.
Fold 3 dimulai...
  Train size: 907, Test size: 227
Akurasi Fold 3: 0.7093
Fold 3 selesai.
Fold 4 dimulai...
  Train size: 907, Test size: 227
Akurasi Fold 4: 0.6740
Fold 4 selesai.
Fold 5 dimulai...
  Train size: 908, Test size: 226
Akurasi Fold 5: 0.7522
Fold 5 selesai.

=== Classification Report ===
              precision    recall  f1-score   support

     Kelas 0       0.74      0.79      0.77       541
     Kelas 1       0.70      0.75      0.73       460
     Kelas 2       0.66      0.31      0.42       133

    accuracy                           0.72      1134
   macro avg       0.70      0.62      0.64      1134
weighted avg       0.72      0.72      0.71      1134

=== Confusion Matrix ===
[[429 103   9]
 [101 347  12]
 [ 48  44  41]]
Cohen's Kappa Score: 0.5128
Akurasi Rata-rata: 0.7205

Akurasi

In [1]:
import joblib
from collections import defaultdict
import math